<a href="https://colab.research.google.com/github/audrb1999/ML-DL-Study/blob/main/Tensorflow/%ED%8A%9C%ED%86%A0%EB%A6%AC%EC%96%BC/06.%20tensorflow_prunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 자습서에서는 다음을 수행합니다.

1. MNIST 용 tf.keras 모델을 처음부터 훈련합니다.
가지 치기 API를 적용하여 모델을 미세 조정하고 정확도를 확인하십시오.

2. 가지 치기에서 3 배 더 작은 TF 및 TFLite 모델을 만듭니다.

3. 가지 치기와 훈련 후 양자화를 결합하여 10 배 더 작은 TFLite 모델을 만듭니다.

4. TF에서 TFLite까지 정확도의 지속성을 확인하십시오.


In [2]:
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 211 kB 3.1 MB/s 


In [3]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

In [6]:
# 데이터 불러오기
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 0~1사이로 정규화 
train_images = train_images/255.0
test_images = test_images/255.0

# 모델 구조 정의
model = keras.Sequential([
                          keras.layers.InputLayer(input_shape = (28, 28)),
                          keras.layers.Reshape(target_shape = (28, 28, 1)),# shape을 바꿈
                          keras.layers.Conv2D(filters = 12, kernel_size = (3, 3), activation = 'relu'),
                          keras.layers.MaxPooling2D(pool_size = (2, 2)),
                          keras.layers.Flatten(),
                          keras.layers.Dense(10)
])

# 모델 학습
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

model.fit(
    train_images, train_labels, epochs = 4, validation_split=0.1
)




Epoch 1/4
1688/1688 [==============================] - 20s 11ms/step - loss: 0.2828 - accuracy: 0.9201 - val_loss: 0.1180 - val_accuracy: 0.9717
Epoch 2/4
1688/1688 [==============================] - 19s 11ms/step - loss: 0.1153 - accuracy: 0.9682 - val_loss: 0.0846 - val_accuracy: 0.9778
Epoch 3/4
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0853 - accuracy: 0.9754 - val_loss: 0.0854 - val_accuracy: 0.9757
Epoch 4/4
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0699 - accuracy: 0.9799 - val_loss: 0.0657 - val_accuracy: 0.9828


In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 12)        120       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 12)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2028)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20290     
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
_________________________________________________________________


In [8]:
_, base_acc = model.evaluate(test_images, test_labels, verbose = 0)

print('base_acc', base_acc)

#저장
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

print('saved baseline model to', keras_file)

base_acc 0.9785000085830688
saved baseline model to /tmp/tmp6qzs1e1k.h5


In [9]:
# 가지 치기로 모델 미세 조정

import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 128
epochs = 2
validation_split = 0.1 # 10%만 validation set으로 사용

num_images = train_images.shape[0] * (1 - validation_split) # 학습 데이터 
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs # 몇 step을 하는지

# Pruning 정의

pruning_params = {
    'pruing_schedule' : tfmot.sparsity.keras.PolynomialDecay(initial_sparsity = 0.5,
                                                             final_sparsity = 0.8,
                                                             begin_step = 0,
                                                             end_step = end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# compile를 다시 설정해줘야함

model_for_pruning.compile(optimizer = 'adam',
                          loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                          metrics = ['accuracy'])

model_for_pruning.summary()


/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape_ (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten_ (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense_1  (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


In [10]:
_, prune_acc = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', base_acc) 
print('Pruned test accuracy:', prune_acc)

Baseline test accuracy: 0.9785000085830688
Pruned test accuracy: 0.9785000085830688


In [13]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 15s 32ms/step - loss: 0.0630 - accuracy: 0.9823 - val_loss: 0.0651 - val_accuracy: 0.9835
Epoch 2/2
422/422 [==============================] - 13s 32ms/step - loss: 0.0558 - accuracy: 0.9844 - val_loss: 0.0642 - val_accuracy: 0.9827


In [15]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', base_acc) 
print('Pruned test accuracy:', prune_acc)

Baseline test accuracy: 0.9785000085830688
Pruned test accuracy: 0.9785000085830688


# 가지 치기를 통해 3 배 더 작은 모델 생성

정리의 압축 이점을 확인하려면 tfmot.sparsity.keras.strip_pruning 과 표준 압축 알고리즘 (예 : gzip을 통해) 적용이 필요합니다.

- strip_pruning 은 학습 중에 만 필요로하는 모든 tf.Variable을 제거하기 때문에 필요합니다. 그렇지 않으면 추론 중에 모델 크기가 추가됩니다.


- 직렬화 된 가중치 행렬은 가지 치기 전과 크기가 같으므로 표준 압축 알고리즘을 적용해야합니다. 그러나 가지 치기는 대부분의 가중치를 0으로 만들고 알고리즘이 모델을 추가로 압축하는 데 사용할 수있는 중복성을 추가합니다.

In [16]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)# 압축 알고리즘 사용

# 저장
_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /tmp/tmp4ke7rb82.h5


In [17]:
# TFlite용 압축 가능한 모델을 만듭니다.

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)# TFLite로 변경할 주소
pruned_tflite_model = converter.convert()# 변경

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmptdxnopda/assets
Saved pruned TFLite model to: /tmp/tmpotxxox1z.tflite


In [18]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [19]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 78247.00 bytes
Size of gzipped pruned Keras model: 48479.00 bytes
Size of gzipped pruned TFlite model: 47736.00 bytes


#가지 치기와 양자화를 결합하여 10 배 더 작은 모델 생성

추가 이점을 위해 정리 된 모델에 사후 훈련 양자화를 적용 할 수 있습니다.

In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmp9ym5ul5r/assets


INFO:tensorflow:Assets written to: /tmp/tmp9ym5ul5r/assets


Saved quantized and pruned TFLite model to: /tmp/tmp0fg5b1i5.tflite
Size of gzipped baseline Keras model: 78247.00 bytes
Size of gzipped pruned and quantized TFlite model: 13722.00 bytes


# TF에서 TFLite까지 정확도 지속성 확인
테스트 데이터 세트에서 TF Lite 모델을 평가하는 도우미 함수를 정의합니다.

In [23]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

프루닝 및 양자화 모델을 평가하고 TensorFlow의 정확도가 TFLite 백엔드까지 유지되는지 확인합니다.

In [24]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Pruned and quantized TFLite test_accuracy: 0.9792
Pruned TF test accuracy: 0.9793000221252441
